# Probability Distribution using VQCs / Quantum Machine Learning TODO

- Goal: Create a pipeline to generate probability distributions using VQCs
- Method: 
    1. Define number of qubits, layers and a quantum circuit
    2. Generate Training Data for variable function
    3. Train the circuit (with cost function) to fit into the target function
    4. Collect many outputs from the circuit to generate a distribution (stochastic process)
    5. Plot the distribution
- DOESNT WORK NO TODO

In [ ]:
# import libraries
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt

# Define the number of qubits and layers (starting slow)
num_qubits = 1
num_gates = 5

# Define the quantum device
dev = qml.device("default.qubit", wires=num_qubits)
# Define the variational circuit
@qml.qnode(dev)
def circuit(weights, x):
    qml.RY(weights[0] * x, wires=0)
    qml.RY(weights[1] * x, wires=0)
    qml.RY(weights[2], wires=0)
    qml.RY(weights[3], wires=0)
    qml.RY(weights[4] * x, wires=0)
    
    # Encode the input x into the circuit (manually or using templates)
    # for counter in range(num_gates):
    #     qml.RY(weights[counter] * x, wires=0)
        #qml.RZ(x * weights[counter], wires=0)
    
    # qml.templates.AngleEmbedding(x, wires=range(num_qubits))
    # qml.templates.StronglyEntanglingLayers(weights * x, wires=range(num_qubits))
    return qml.expval(qml.PauliZ(0))

# Generate training data for a function (e.g. sin(x), x^2, sin(x) + x^2, etc.)
num_training_points = 250
input_train = np.random.uniform(0, 2*np.pi, num_training_points) # random values

# Define the target function
sin_x = np.sin(input_train)
sin_x_noise = sin_x + 0.1 * np.random.normal(size=num_training_points)
target_function = sin_x


# Initialize the weights and optimizer
weights = np.random.uniform(0, 2 * np.pi, num_gates)
opt = qml.AdamOptimizer(stepsize=0.01)
steps = 200

# Train the circuit

# Define some loss functions
def mean_squared_error(prediction, target):
    return ((prediction - target)**2) / 2

# Define the cost function
def cost(weights, idx):
    x = input_train[idx]
    target = target_function[idx]
    prediction = circuit(weights, x)
    regularization = np.sum(np.abs(weights))
    return mean_squared_error(prediction, target) + 0.01 * regularization

for i in range(steps):
    for idx in range(num_training_points):
        weights = opt.step(cost, weights, idx=idx)
    if (i + 1) % 5 == 0:
        # for x in range(input_train.shape[0]):
        #     predicted_output = circuit(weights, x)
        #     error = np.abs(predicted_output - target_function[x])
        #     print(f"Step: {i}, Input: {x}, Expected: {target_function[x]:.4f}, Predicted: {predicted_output:.4f}, Error: {error:.4f}")
        predicted_output = circuit(weights, idx)
        error = np.abs(predicted_output - target_function[idx])
        # print(f"Cost after step {i + 1}/{steps}: {cost(weights, idx=idx)}\n target: {target_function[idx]}, prediction: {predicted_output}, error: {error}")
        print(f"Cost after step {i + 1}/{steps}: {cost(weights, idx=idx)}")


# Plot the predictions
# Generate input values for the plot
input_values = np.array(input_train)

# Compute actual function values and predicted function values
predicted_values = np.array([circuit(weights, x) for x in input_values])

# Sort the values for plotting
sort_indices = np.argsort(input_values)
input_values = input_values[sort_indices]
predicted_values = predicted_values[sort_indices]
target_function = target_function[sort_indices]

# Plot actual function values and predicted function values
plt.figure(figsize=(10, 6))
plt.plot(input_values, target_function, label='Actual function')
plt.plot(input_values, predicted_values, label='Predicted function')
plt.legend()
plt.xlabel('Input')
plt.ylabel('Function values')
plt.title('Actual vs Predicted function')
plt.show()

In [ ]:
# Collect many outputs from the circuit to generate a distribution
num_shots = 100
output_states = []
for i, x in enumerate(input_train):
    for _ in range(num_shots):
        output_states.append(circuit(weights, x=x))
    if i % 10 == 0:  # Adjust this value based on the size of your input_train
        print(f"Processed {i} inputs out of {len(input_train)}")

In [ ]:
# Plot the distribution
plt.hist(output_states, bins=30, density=True)
plt.xlabel("Output state")
plt.ylabel("Probability")
plt.show()